In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('../input/multilang-org-dataset/data.csv')
df.head()

,name_1,name_2,is_dup
0,"Pu Max Trading (T) Co., Ltd.","Т.А. Цорпоратион Пвт., Лтд.",0
1,Схалфин Традинг Цо. Wлл,Dhaka Transport And Trading.,0
2,SOPREMA GmbH,СОПРЕМА ГмбХ,1
3,Kuraray India Private Ltd.,Spiro Hoses India Private Ltd.,0
4,Вибрацоустиц Индиа Привате Лтд.,Wictor Compounding (India) Private Ltd.,0


In [3]:
import tensorflow
from tensorflow.keras import backend as K

In [4]:
def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
from sklearn.model_selection import train_test_split

train, test  = train_test_split(df,
                                test_size=0.1, 
                                random_state=128,
                                stratify=df['is_dup'])

In [6]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 828.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b50755337d8d682c8cddbd6eda227dedeb7353f8ec6f0fb86f27abd23dffdce4
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence_transformers


In [7]:
from sentence_transformers import evaluation
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from tqdm import tqdm

In [8]:
ls = train.values.tolist()
train_examples = []
for x in tqdm(range(len(train))):
    train_examples.append(InputExample(texts=[ls[x][0], ls[x][1]], label=float(ls[x][2])))

100%|██████████| 263376/263376 [00:01<00:00, 152559.74it/s]


In [9]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [10]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', cache_folder='./')

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [11]:
model = SentenceTransformer('./sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2', device = 'cuda')

In [12]:
train_loss = losses.CosineSimilarityLoss(model)

In [13]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16461 [00:00<?, ?it/s]

In [14]:
from sentence_transformers import util

In [15]:
# Two lists of sentences
name_1 = [x[0] for x in test[['name_1']].values.tolist()]
name_2 = [x[0] for x in test[['name_2']].values.tolist()]

#Compute embedding for both lists
embeddings1 = model.encode(name_1, convert_to_tensor=True)
embeddings2 = model.encode(name_2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)


Batches:   0%|          | 0/915 [00:00<?, ?it/s]

Batches:   0%|          | 0/915 [00:00<?, ?it/s]

In [16]:
y_pred = []
for i in range(len(cosine_scores)):
    y_pred.append(float(cosine_scores[i][i]))

In [17]:
y_predf = [1 if x > 0.7 else 0 for x in y_pred]

In [18]:
y_test = test.is_dup.values.tolist()

In [19]:
y_predf = np.asarray(y_predf, dtype='float32')
y_test = np.asarray(y_test, dtype='float32')

In [20]:
score = f1(y_test, y_predf)

2022-10-16 19:51:16.891278: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 19:51:16.892268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 19:51:16.892858: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 19:51:16.896746: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [21]:
score.numpy()

0.99986327

In [22]:
def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [23]:
rec = recall(y_test, y_predf)
pre = precision(y_test, y_predf)
print('Crazy Metrics')
print('-------------')
print(f'Recall: {rec}')
print('-------------')
print(f'Precision: {pre}')
print('-------------')
print(f'F1-score: {score}')

Crazy Metrics
-------------
Recall: 0.9997949600219727
-------------
Precision: 0.9999316334724426
-------------
F1-score: 0.9998632669448853


# Расстояние Левенштейна

In [24]:
def levenstein(str_1, str_2):
    n, m = len(str_1), len(str_2)
    if n > m:
        str_1, str_2 = str_2, str_1
        n, m = m, n

    current_row = range(n + 1)
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = previous_row[j] + 1, current_row[j - 1] + 1, previous_row[j - 1]
            if str_1[j - 1] != str_2[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    return current_row[n]

In [25]:
levenstein('Gazprom', 'Газпром')

7

In [26]:
def get_y_from_levenstein(name1,name2):
    y = []
    for x in range(len(name1)):
        score = levenstein(name1[x],name2[x])
        if score <=0:
            y.append(1)
        else:
            y.append(0)
    return y

In [27]:
y_leven = get_y_from_levenstein(name_1,name_2)
y_leven = np.asarray(y_leven, dtype='float32')

In [28]:
scorel = f1(y_test, y_leven)
recl = recall(y_test, y_leven)
prel = precision(y_test, y_leven)
print('Levenstein Metrics')
print('-------------')
print(f'Recall: {recl}')
print('-------------')
print(f'Precision: {prel}')
print('-------------')
print(f'F1-score: {scorel}')

Levenstein Metrics
-------------
Recall: 0.11919081211090088
-------------
Precision: 1.0
-------------
F1-score: 0.21299460530281067


In [29]:
gazprom_ru = ['Газпром ООО']
gazprom_en = ['OAO Gazprom']

emb1 = model.encode(gazprom_ru, convert_to_tensor=True)
emb2 = model.encode(gazprom_en, convert_to_tensor=True)

gaz = util.cos_sim(emb1, emb2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
gaz[0][0]

tensor(0.9827, device='cuda:0')

In [31]:
model.save('./trained_MiniLM-L12-v2')

In [32]:
import shutil
shutil.make_archive('./trained_MiniLM-L12-v2', 'zip', './trained_MiniLM-L12-v2')

'/kaggle/working/trained_MiniLM-L12-v2.zip'